In [1]:
# # Run it once only
# !pip install langchain
# !pip install sentence_transformers
# !pip install chromadb
# !pip install numpy
# !pip install pandas
# !pip install tensorflow
# !pip install ipywidgets
# !pip install pandas
# !pip install chainlit

In [2]:
import pandas as pd

In [3]:
!ollama list

NAME    	ID          	SIZE  	MODIFIED   
gemma:2b	b50d6c999e59	1.7 GB	7 days ago	


# 1. Loading the LLM : Gemma

In [4]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms.ollama import Ollama

llm = Ollama(
    base_url="http://localhost:11434",
    model="gemma:2b",
    callback_manager=CallbackManager(
        [StreamingStdOutCallbackHandler()],
    ),
    temperature = 0.0,
)

# # Testing of the Gemma
# while True:
#     query = input("Enter a query: ")
#     if(query.lower()=="exit"):
#         break
#     llm.invoke(query)
#     print()

In [5]:
# import os
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# def process_text_files(directory):
#     # Initialize the text splitter with desired parameters
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1500,
#         chunk_overlap=150,
#     )

#     # Process each .txt file in the directory
#     for filename in os.listdir(directory):
#         if filename.endswith(".txt"):
#             txt_file_path = os.path.join(directory, filename)
            
#             with open(txt_file_path, "r", encoding="utf-8") as file:
#                 text = file.read()
            
#             # Split the text into chunks using the text splitter
#             splits = text_splitter.split_text(text)

#             # Add some indexing to the splits
#             splits_with_page_no = []
#             for i, split in enumerate(splits):
#                 split = f"page : {i+1}\n\n" + split
#                 splits_with_page_no.append(split)

#             # For example, print the first 5 pages of each file
#             print(f"Processed {filename}:")
#             for page in splits_with_page_no[:5]:
#                 print(page)
#                 print("-----"*20)  # Separator between pages for clarity
                
#     return splits_with_page_no

# # Specify the directory containing the text files
# directory = "Books/Texts/"
# splits_with_page_no = process_text_files(directory)


In [6]:
# # Create the vector database : Run it once only
# # !rmdir ./database/ # Remove old database file if any

# from langchain.vectorstores import Chroma


# persist_directory = "database/"
# from langchain_community.embeddings.sentence_transformer import (
#     SentenceTransformerEmbeddings,
# )
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2") # Different embedding function

# # For text files
# vectordb = Chroma.from_texts(
#     texts = splits_with_page_no,
#     embedding = embedding_function,
#     persist_directory = persist_directory,
# )

In [7]:
# Load the vector database
from langchain.vectorstores import Chroma
# from .autonotebook import tqdm as notebook_tqdm

persist_directory = "database/"
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2") # Different embedding function

# For text files
vectordb = Chroma(
    embedding_function = embedding_function,
    persist_directory = persist_directory,
)

# Test of LLM

In [8]:
# from langchain.output_parsers import ResponseSchema, StructuredOutputParser
# from langchain.prompts import ChatPromptTemplate

# # Define response schemas
# brand_name_schema = ResponseSchema(name="brand_name",
#                                    description="This is the name of the brand")
# likelihood_of_success_schema = ResponseSchema(name="likelihood_of_success",
#                                               description="This is an integer score between 1-10")
# reasoning_schema = ResponseSchema(name="reasoning",
#                                    description="This is the reasons for the score")

# response_schemas = [brand_name_schema, likelihood_of_success_schema, reasoning_schema]

# # Create structured output parser
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# # Get format instructions
# format_instructions = output_parser.get_format_instructions()

# # Define template string
# template_string = """You are a master branding consultant who specializes in naming brands. \
# You come up with catchy and memorable brand names.

# Take the brand description below delimited by triple backticks and use it to create the name for a brand.

# brand description: ```{brand_description}```

# Then based on the description and your hot new brand name give the brand a score 1-10 for how likely it is to succeed.

# {format_instructions}
# """

# # Create chat prompt template
# prompt = ChatPromptTemplate.from_template(template=template_string)

# # Format messages
# messages = prompt.format_messages(brand_description="This brand makes shoes for rich kids. It's shoes are upto current fashion standards.", 
#                                   format_instructions=format_instructions)

# # Invoke the language model
# response = llm.invoke(messages)

# # Modified response
# modified_response = response.split("```json\n")[-1].split("```")[0]

# # Extract structured output
# structured_output = output_parser.parse(modified_response)

# # Print structured output as JSON
# print(structured_output)

In [9]:
# # Extract structured output
# structured_output = output_parser.parse(response)

# # Print structured output as JSON
# print(type(structured_output))

# Patient Details Extraction

In [10]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import OutputFixingParser

def ask_patient_info():
    # Define response schemas
    patient_name_schema = ResponseSchema(name="patient name",
                                         description="This is the name of the patient",
                                        )
    patient_age_schema = ResponseSchema(name="age",
                                        description="This is the age of the patient",
                                       )
    patient_gender_schema = ResponseSchema(name="gender",
                                           description="This is the gender (Male or Female) of the patient",
                                          )
    patient_height_schema = ResponseSchema(name="height",
                                           description="This is the height of the patient",
                                          )
    patient_weight_schema = ResponseSchema(name="weight",
                                           description="This is the weight of the patient",
                                          )
    patient_symptoms_schema = ResponseSchema(name="symptoms",
                                             description="Descriptions of medical condition that the patient is facing.",
                                            )
    patient_allergies_schema = ResponseSchema(name="allergies",
                                              description="This is the allergies that the patient have.",
                                             )
    response_schemas = [patient_name_schema,
                        patient_age_schema,
                        patient_gender_schema,
                        patient_height_schema,
                        patient_weight_schema,
                        patient_symptoms_schema,
                        patient_allergies_schema,
                       ]
    
    # Create structured output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    
    # Get format instructions
    format_instructions = output_parser.get_format_instructions()
    
    # Define the template string
    template_string = """You are an automatic form-filling system designed to collect patient information. 
    Fill out the following details based on the user's response below. Write "None" if a particular information is missing.:
    
    user response: ```{user_response}```
    
    {format_instructions}
    """
    
    # Create chat prompt template
    prompt = ChatPromptTemplate.from_template(template=template_string)
    
    # Formate messages
    # # Example 1:
    # user_response = """Hi, I am Anjum Ul Azim. I am 24 years old. I am a male of course. I have back pain and fever. Also I have headache and mild fever. I don't have
    # any kind of allergies. My height is 169 cm. I weigh 54.5 kg.
    # """
    # # Example 2:
    # user_response = """Hi, I am Naman Sethiya. I am 20 years old. I am a male of course. I have knee pain. I am allergic to peanuts.
    # My height is 171 cm. I weigh 65.5 kg.
    # """
    # # Example 3:
    # user_response = """Hi, I am Adarsh Sharma. I am 20 years old. I am a male of course. I am having low-grade fevers, malaise, weight loss, myalgias, and arthralgias. I am allergic to meat.
    # My weight is 50 kg. I am 169 cm tall and I have no allergies.
    # """
    
    # User input
    user_response = input("User : ")
    
    messages = prompt.format_messages(user_response=user_response,
                                      format_instructions=format_instructions,
                                     )
    
    # Invoke the language model
    response = llm.invoke(messages)
    
    
    print(response)
    # Extract the structured output
    try:
        structured_output = output_parser.parse(response)
        # print(structured_output)
        # print("\nParsing Successful")
    except:
        # print("There is some error in the parsing")
        new_parser = OutputFixingParser.from_llm(parser=output_parser,
                                                 llm=llm,
                                                )
        structured_output = new_parser.parse(response)
        # print(structured_output)
        # print("\nParsing Successful")
    return structured_output


In [11]:
# # Fill the remaining details untill it's full.
# from langchain.output_parsers import ResponseSchema, StructuredOutputParser
# from langchain.prompts import ChatPromptTemplate

# # Define response schemas
# system_response_schema = ResponseSchema(name="system query",
#                                         description="""System query asking the patient for the information missing.""",
#                                        )
# response_schemas = [system_response_schema]
# # Create structured output parser
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# # Get format instructions
# format_instructions = output_parser.get_format_instructions()

# # Define the template string
# template_string = """You are an automatic form-filling system designed to collect patient information. 
# Ask the user for the information only that is missing (indicated by "None"):

# user information: ```{user_data}```

# {format_instructions}
# """

# # Create chat prompt template
# prompt = ChatPromptTemplate.from_template(template=template_string)

# # Formate messages
# # # Example 1:
# # user_response = """Hi, I am Anjum Ul Azim. I am 24 years old. I am a male of course. I have back pain and fever. Also I have headache. I don't have
# # any kind of allergies. My height is 169 cm. I weigh 54.5 kg.
# # """
# # # Example 2:
# # user_response = """Hi, I am Naman Sethiya. I am 20 years old. I am a male of course. I have knee pain. I am allergic to peanuts.
# # My height is 171 cm. I weigh 65.5 kg.
# # """
# # Example 3:
# user_response = """Hi, I am Adarsh Sharma. I am 20 years old. I am a male of course. I have obsessive compulsive disorder. I am allergic to meat.
# My weight is 50 kg.
# """
# # # User input
# # user_response = input("User: ")

# # Example 1:
# user_data = """```json
# {
# 	"patient name": "None",
# 	"age": "20",
# 	"gender": "Male",
# 	"height": "None",
# 	"weight": "50 kg",
# 	"symptoms": "Obsessive compulsive disorder",
# 	"allergies": "Meat allergies"
# }
# ```"""

# # user_data = response
# print(user_data)

# messages = prompt.format_messages(user_data=user_data,
#                                   format_instructions=format_instructions,
#                                  )

# # Invoke the language model
# response2 = llm.invoke(messages)



In [12]:
# # Fill the remaining details untill it's full.
# from langchain.output_parsers import ResponseSchema, StructuredOutputParser
# from langchain.prompts import ChatPromptTemplate


# # Define the template string
# template_string = """
# user information: ```{user_data}```

# ask the user for the missing information given above like "what is your name?", "What is your age?" etc if it is missing in above data.

# """

# # Create chat prompt template
# prompt = ChatPromptTemplate.from_template(template=template_string)

# # Formate messages
# # Example 1:
# user_data = """```json
# {
# 	"patient name": "None",
# 	"age": "20",
# 	"gender": "Male",
# 	"height": "None",
# 	"weight": "50 kg",
# 	"symptoms": "Obsessive compulsive disorder",
# 	"allergies": "Meat allergies"
# }
# ```"""

# # user_data = response
# print(user_data)

# messages = prompt.format_messages(user_data=user_data)

# # Invoke the language model
# response2 = llm.invoke(messages)

In [13]:
# Fill in the remaining details


In [14]:
# Find diseases
from langchain.chains import RetrievalQA

def find_diseases(patient_symptoms):
    question_template = """List down most possible diseases or disorders based on the patients symptoms given below :
    
    Patient Symptoms : {patient_symptoms}
    """
    
    question = question_template.format(patient_symptoms=patient_symptoms)
    
    # # This method consumes too much resources
    # qa_chain_mmr = RetrievalQA.from_chain_type(
    #     llm,
    #     retriever = vectordb.as_retriever(),
    #     chain_type="map_reduce"
    # )
    
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(),
    )
    result = qa_chain.invoke({"query": question})

    # Format Disease List
    disease_list = result["result"]
    # Define response schemas
    possible_diseases_schema = ResponseSchema(name="possible diseases",
                                         description="Comma separated top ten most probable list of diseases",
                                        )
    response_schemas = [possible_diseases_schema]
    
    # Create structured output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    
    # Get format instructions
    format_instructions = output_parser.get_format_instructions()
    
    # Define the template string
    template_string = """Convert the list of diseases given below to the format as needed.
    
    Disease List :
    {disease_list}
    
    {format_instructions}
    """
    
    # Create chat prompt template
    prompt = ChatPromptTemplate.from_template(template=template_string)
    
    # # Formate messages
    # # Example 1:
    # disease_list = """Sure, here are the possible diseases or disorders based on the patient's symptoms:
    
    # 1. Leukocytoblastic vasculitis
    # 2. Renal disease
    # 3. Polyarteritis nodosa
    # 4. Systemic lupus erythematosus
    # 5. Rheumatoid arthritis
    # 6. Scleroderma
    # 7. Polymyositis
    # 8. Limited Wegener's disease"""
    
    messages = prompt.format_messages(disease_list=disease_list,
                                      format_instructions=format_instructions,
                                     )
    
    # Invoke the language model
    response = llm.invoke(messages)

    
    # Extract the structured output
    try:
        structured_output = output_parser.parse(response)
        # print(structured_output)
        # print("\nParsing Successful")
    except:
        # print("There is some error in the parsing")
        new_parser = OutputFixingParser.from_llm(parser=output_parser,
                                                 llm=llm,
                                                )
        structured_output = new_parser.parse(response)
        # print(structured_output)
        # print("\nParsing Successful")

    disease_list = structured_output["possible diseases"].split(",")
    
    return disease_list

In [15]:
def get_treatment(disease):
    template_string = """What is the treatment for {disease}?"""
    template_string = template_string.format(disease=disease)
    result = llm.invoke(template_string)
    return result
# result = get_treatment("Common Cold")
    

def get_more_treatments(disease_list, size=1):
    result = []
    for i in range(size):
        print(f"\nResult {i+1} :\n")
        result.append(get_treatment(disease_list[i]))
    return result

In [16]:
# print(result["result"])

In [17]:
def format_info(structured_output, disease_list, treatments):
    df1 = pd.DataFrame([structured_output])
    temp_dict = {}
    for i in range(3):
        temp_str = "***Disease : "+disease_list[i]+"\n\n"+treatments[i]
        temp_dict[f"disease{i+1}"] = temp_str
    df2 = pd.DataFrame([temp_dict])
    df = pd.concat([df1, df2], axis=1)
    return df

In [18]:
def add_info(structured_output, disease_list, treatments, user_database, database_path):
    df = format_info(structured_output, disease_list, treatments)
    updated_user_database = pd.concat([df, user_database], axis=0, ignore_index=True)
    updated_user_database.to_csv(database_path, index=False)
    return updated_user_database

In [19]:
def chat(user_database, database_path):
    # Ask Patient's info
    structured_output = ask_patient_info()
    # Find Possible Diseases
    patient_symptoms = structured_output["symptoms"]
    patient_allergies = structured_output["allergies"]
    disease_list = find_diseases(patient_symptoms)
    # Get treatments
    treatments = get_more_treatments(disease_list, size=3)
    user_database = add_info(structured_output, disease_list, treatments, user_database, database_path)
    return user_database

In [20]:

database_path = "example_user_database.csv"

def continuous_chat():
    user_database = pd.read_csv(database_path)
    for i in range(10):
        user_database = chat(user_database, database_path)
        query = input("Do you have any other query (y/n) :  ")
        if query.lower() == "y":
            continue
        else:
            break
    user_database.to_csv(database_path, index=False)
    

In [ ]:
continuous_chat()


User :  hi , I am rajat. i am a male and of 21 years old. sincee a year I have a case of cough and runny nose and cough which doesnt seem to go away with anti allergy medications


```json
{
	"patient name": "Rajat",
	"age": "21",
	"gender": "Male",
	"height": "None",
	"weight": "None",
	"symptoms": "Cough and runny nose, cough that doesn't go away with anti allergy medications",
	"allergies": "None"
}
``````json
{
	"patient name": "Rajat",
	"age": "21",
	"gender": "Male",
	"height": "None",
	"weight": "None",
	"symptoms": "Cough and runny nose, cough that doesn't go away with anti allergy medications",
	"allergies": "None"
}
```
The patient symptoms suggest the presence of allergy, asthma, and autoimmune diseases.

Possible diseases or disorders based on the patient symptoms are:

- Wegener's granulomatosis
- Multiple endocrine neoplasia syndromes
- Rheumatoid arthritis
- Psoriasis
- Lupus
- Crohn's disease
- Fibromyalgia
- Multiple sclerosis
- Guillain-Barré syndrome
- Vitiligo
- Scleroderma
- Psoriatic arthritis
- Rheumatoid arthritis
- Lupus
- Crohn's disease
- Fibromyalgia
- Multiple sclerosis
- Guillain-Barré syndrome
- Vitiligo
- Scleroderma```json
{
	"pos

In [21]:
# Example 1:
user_response = """Hi, I am Mukesh. I am 35 years old. I am a male of course. I have nausia, dizziness, hea. I do not have
any kind of allergies. My height is 169 cm. I weigh 53.2 kg.
"""
# Example 2:
user_response = """Hi, I am Rakesh Sharma. I am 20 years old. I am a male of course. I am having low-grade fevers, malaise, weight loss, myalgias, and arthralgias. I am allergic to meat.
My weight is 50 kg. I am 169 cm tall and I have no allergies.
"""
# Example 1:
user_response = """Hi, I am Devanadika. I am 19 years old. I am female. I have stomach ache, fever, headache. I don't have
any kind of allergies. My height is 161 cm. I weigh 51 kg.
"""

In [22]:
# from langchain.prompts import ChatPromptTemplate
# from langchain.schema import StrOutputParser
# from langchain.schema.runnable import Runnable
# from langchain.schema.runnable.config import RunnableConfig

# import chainlit as cl


# @cl.on_chat_start
# async def on_chat_start():
#     model = llm
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#                 "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions.",
#             ),
#             ("human", "{question}"),
#         ]
#     )
#     runnable = prompt | model | StrOutputParser()
#     cl.user_session.set("runnable", runnable)


# @cl.on_message
# async def on_message(message: cl.Message):
#     runnable = cl.user_session.get("runnable")  # type: Runnable

#     msg = cl.Message(content="")

#     async for chunk in runnable.astream(
#         {"question": message.content},
#         config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
#     ):
#         await msg.stream_token(chunk)

#     await msg.send()